# Import
This first block is only for starting in Colabratory. Some pips need to be re-installed each time and authentication codes need to be entered from links to allow access to google drive files. Kind of annoying but free GPU and such (be sure to tic it on in notebook settings under edit). Remove this block if not using Colabratory.

In [0]:
from google.colab import auth
auth.authenticate_user()
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras
!pip install numba

gpg: keybox '/tmp/tmplxmg8_mv/pubring.gpg' created
gpg: /tmp/tmplxmg8_mv/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [0]:
# from datetime import datetime 
# start_real = datetime.now()
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Dense, concatenate, Embedding, Flatten, Activation, SpatialDropout1D
from keras.layers import Bidirectional, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.models import Model

from nltk.corpus import stopwords
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM, CuDNNGRU
from keras.layers import MaxPooling1D, Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'

import re
import math
# set seed
np.random.seed(123)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


This block is to set up and/or alter GPU settings. Remove if not using GPU

In [0]:
# GPU acceleration
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
# import theano
# don't overload gpu
from keras import backend as K
config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
session = K.tf.Session(config=config)

# Read in data
Change this to target where you keep your train and test data.

In [0]:
train = pd.read_csv('drive/Donor/train.csv')
test = pd.read_csv('drive/Donor/test.csv')

In [0]:
print(train.shape[0])
print(test.shape[0])
train[:10]

182080
78035


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1
5,p063374,403c6783e9286e51ab318fba40f8d729,Mrs.,DE,2016-11-05 10:01:51,Grades PreK-2,"Applied Learning, Literacy & Language","Character Education, Literature & Writing",Need to Reach Our Virtual Mentors!!!,My kids tell me each day that they want to mak...,I started a program called Telementoring in ho...,NaN,NaN,My students need tablets in order to communic...,0,1
6,p103285,4e156c5fb3eea2531601c8736f3751a7,Mrs.,MO,2016-08-31 00:30:43,Grades PreK-2,Health & Sports,Health & Wellness,Active Kindergartners,Kindergarten is the new first grade. My studen...,With balance discs and stools as flexible seat...,NaN,NaN,My students need stability stools and inflatab...,1,1
7,p181781,c71f2ef13b4bc91afac61ca8fd4c0f9f,Mrs.,SC,2016-08-03 13:26:01,Grades PreK-2,"Applied Learning, Literacy & Language","Early Development, Literature & Writing",Fabulous Firsties-Wiggling to Learn!,First graders are fantastic! They are excited ...,First graders love learning! We need 6 wiggle-...,NaN,NaN,My students need wiggle stools to allow them t...,0,1
8,p114989,b580c11b1497a0a67317763b7f03eb27,Ms.,IN,2016-09-13 22:35:57,Grades 6-8,Math & Science,Mathematics,Wobble Chairs Help Fidgety Kids Focus,My seventh graders dream big. They can't wait ...,I have used alternative seating in my classroo...,NaN,NaN,My students need seating that allows the most ...,13,1
9,p191410,2071fb0af994f8f16e7c6ed0f35062a1,Mrs.,IL,2016-09-24 18:38:59,Grades PreK-2,Literacy & Language,Literacy,Snuggle Up With A Good Book,I teach first grade in a small farming town in...,There is nothing better than snuggling up with...,NaN,NaN,My students need 2 youth sized reclining chair...,12,1


Split the training set into a train and development set.

In [0]:
train, dev = train_test_split(train, random_state=233, train_size=0.9)

n_train = train.shape[0]
n_test = test.shape[0]
n_dev = dev.shape[0]

print("Training on", n_train, "examples")
print("Validating on", n_dev, "examples")
print("Testing on", n_test, "examples")

Training on 163872 examples
Validating on 18208 examples
Testing on 78035 examples


In [0]:
# Concatenate train - dev - test data for easy to handle
full_df = pd.concat([train, dev, test])
full_df = full_df.fillna('MISSING VALUE')

In [0]:
full_df[:5]

,id,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_grade_category,project_is_approved,project_resource_summary,project_subject_categories,project_subject_subcategories,project_submitted_datetime,project_title,school_state,teacher_id,teacher_number_of_previously_posted_projects,teacher_prefix
99111,p195591,"At MJMS, we're striving to get our middle scho...",Our 6th graders will be able to enjoy a timele...,MISSING VALUE,MISSING VALUE,Grades 6-8,1,My students need A class set of Lois Lowery's ...,Literacy & Language,"ESL, Literacy",2016-08-04 11:32:53,MJMS Cougar 6th graders ready to Number the St...,TX,e15c63c6de4eb2aed25c55e63db446af,0,Mr.
41784,p082205,"When you open a school, you forget sometimes a...",My school needs sets of multicultural books th...,MISSING VALUE,MISSING VALUE,Grades 3-5,1,My students need multicultural books so they c...,Literacy & Language,"Literacy, Literature & Writing",2016-08-09 21:15:55,Well Rounded Reading,NC,e4ab59578a5f3c0d14c1316d9c3a945c,10,Ms.
141032,p183543,My students are all attending a Title one scho...,It is an everyday challenge to level the playi...,MISSING VALUE,MISSING VALUE,Grades PreK-2,1,My students need more IPads to be successful r...,"Literacy & Language, Special Needs","Literacy, Special Needs",2016-10-01 09:50:59,Learning Through iPad Reading,TX,b5e1513ea21294f8ed6688ca2ef1dd1a,7,Mrs.
157388,p098121,My students are above and beyond! These 20 lit...,Moving to Georgia meant more students per clas...,MISSING VALUE,MISSING VALUE,Grades PreK-2,1,My students need classroom essentials such as ...,Literacy & Language,Literature & Writing,2016-08-12 19:32:41,Super Storage for 2nd Grade!,GA,4bb39b32467949aa376027dba589cf86,2,Mrs.
159970,p131924,The students at my school are some of the most...,"The current construction tools are old, well w...",MISSING VALUE,MISSING VALUE,Grades 9-12,1,My students need compasses and straight edges ...,Math & Science,Mathematics,2016-09-06 13:30:06,Geometry Tools,TX,8e7dea1a23a89be17d9238b61fa79dd3,1,Ms.


# Preprocessing the data

In [0]:
time_cols = ['sub_year', 'sub_month', 'sub_day', 'sub_hour', 'sub_minute', 'sub_dayofweek', 'sub_dayofyear']
def time_stamp_features(df):
    time_df = pd.to_datetime(df['project_submitted_datetime'])
    df['sub_year'] = time_df.apply(lambda x: x.year).astype('str')
    df['sub_month'] = time_df.apply(lambda x: x.month).astype('str')
    df['sub_day'] = time_df.apply(lambda x: x.day).astype('str')
    df['sub_hour'] = time_df.apply(lambda x: x.hour)
    df['sub_minute'] = time_df.apply(lambda x: x.minute)
    df['sub_dayofweek'] = time_df.apply(lambda x: x.dayofweek).astype('str')
    df['sub_dayofyear'] = time_df.apply(lambda x: x.dayofyear).astype('str')
    return df
full_df = time_stamp_features(full_df)

In [0]:
%%time

print("Processing categorical data...")
le = LabelEncoder()
# full_df.category = full_df.category_name
le.fit(full_df.teacher_id)
full_df.teacher_id = le.transform(full_df.teacher_id)

le.fit(full_df.teacher_prefix)
full_df.teacher_prefix = le.transform(full_df.teacher_prefix)

le.fit(full_df.school_state)
full_df.school_state = le.transform(full_df.school_state)

le.fit(full_df.project_grade_category)
full_df.project_grade_category = le.transform(full_df.project_grade_category)

le.fit(full_df.project_subject_categories)
full_df.project_subject_categories = le.transform(full_df.project_subject_categories)

le.fit(full_df.project_subject_subcategories)
full_df.project_subject_subcategories = le.transform(full_df.project_subject_subcategories)

le.fit(full_df.sub_year)
full_df.sub_year = le.transform(full_df.sub_year)

le.fit(full_df.sub_month)
full_df.sub_month = le.transform(full_df.sub_month)

le.fit(full_df.sub_dayofweek)
full_df.sub_dayofweek = le.transform(full_df.sub_dayofweek)

le.fit(full_df.sub_dayofyear)
full_df.sub_dayofyear = le.transform(full_df.sub_dayofyear)

del le

Processing categorical data...
CPU times: user 8.17 s, sys: 86 ms, total: 8.26 s
Wall time: 8.15 s


# Time to Tokenize
This turns the text into numerical tokens. Also note that this step also removes all punctuation, rare words, and non words from the data on it's own.

In [0]:
%%time

Max_words = 40000
print("Transforming text data to sequences...")
raw_text = np.hstack([full_df.project_title.str.lower(),
                      full_df.project_essay_1.str.lower(),
                      full_df.project_essay_2.str.lower(),
                      full_df.project_essay_3.str.lower(),
                      full_df.project_essay_4.str.lower(),
                      full_df.project_resource_summary.str.lower(),
                     ])

print("   Fitting tokenizer...")
tok_raw = Tokenizer(num_words = Max_words,
#                   charaters in this filter string are removed before tokenizing 
                    filters='.!,?"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n')
tok_raw.fit_on_texts(raw_text)


print("   Transforming text to sequences...")
full_df.project_title = tok_raw.texts_to_sequences(full_df.project_title.str.lower())
full_df.project_essay_1 = tok_raw.texts_to_sequences(full_df.project_essay_1.str.lower())
full_df.project_essay_2 = tok_raw.texts_to_sequences(full_df.project_essay_2.str.lower())
full_df.project_essay_3 = tok_raw.texts_to_sequences(full_df.project_essay_3.str.lower())
full_df.project_essay_4 = tok_raw.texts_to_sequences(full_df.project_essay_4.str.lower())
full_df.project_resource_summary = tok_raw.texts_to_sequences(full_df.project_resource_summary.str.lower())



Transforming text data to sequences...
   Fitting tokenizer...
   Transforming text to sequences...
CPU times: user 2min 13s, sys: 2.65 s, total: 2min 15s
Wall time: 2min 13s


In [0]:
Max_essay_length = 125
Max_title_length = 20
Max_length = 125

# full_df.project_title = sequence.pad_sequences(full_df.project_title, maxlen=Max_title_length)
# full_df.project_essay_1 = sequence.pad_sequences(full_df.project_essay_1, maxlen=Max_length)
# full_df.project_essay_2 = sequence.pad_sequences(full_df.project_essay_2, maxlen=Max_length)
# full_df.project_essay_3 = sequence.pad_sequences(full_df.project_essay_3, maxlen=Max_length)
# full_df.project_essay_4 = sequence.pad_sequences(full_df.project_essay_4, maxlen=Max_length)
# full_df.project_resource_summary = sequence.pad_sequences(full_df.project_resource_summary, maxlen=Max_length)

# Define constants to use when define RNN model

In [0]:
%%time
time_cols = ['sub_year', 'sub_month', 'sub_day', 'sub_hour', 'sub_minute', 'sub_dayofweek', 'sub_dayofyear']
def get_rnn_data(dataset):
    X = {
          'teacher_id':np.array(dataset.teacher_id),
          'teacher_prefix':np.array(dataset.teacher_prefix),
          'school_state':np.array(dataset.school_state),
          'project_submitted_datetime': np.array(dataset.project_submitted_datetime),
          'project_grade_category':np.array(dataset.project_grade_category),
          'project_subject_categories':np.array(dataset.project_subject_categories),
          'project_subject_subcategories':np.array(dataset.project_subject_subcategories),
          'project_title':pad_sequences(dataset['project_title'], maxlen=Max_title_length),
          'project_essay_1':sequence.pad_sequences(dataset.project_essay_1, maxlen=Max_length),
          'project_essay_2':sequence.pad_sequences(dataset.project_essay_2, maxlen=Max_length),
          'project_essay_3':sequence.pad_sequences(dataset.project_essay_3, maxlen=Max_length),
          'project_essay_4':sequence.pad_sequences(dataset.project_essay_4, maxlen=Max_length),
          'project_resource_summary':sequence.pad_sequences(dataset.project_resource_summary, maxlen=Max_length),
          'teacher_number_of_previously_posted_projects':np.array(dataset.teacher_number_of_previously_posted_projects),
          'sub_year': np.array(dataset['sub_year']),
          'sub_month': np.array(dataset['sub_month']),
          'sub_day': np.array(dataset['sub_day']),
          'sub_hour': np.array(dataset['sub_hour']),
          'sub_minute': np.array(dataset['sub_minute']),
          'sub_dayofweek': np.array(dataset['sub_dayofweek']),
          'sub_dayofyear': np.array(dataset['sub_dayofyear']),
    }
    return X
# X = ['teacher_id',
#      'teacher_prefix',
#      'school_state',
# #      'project_submitted_datetime',
#      'project_grade_category',
#      'project_subject_categories',
#      'project_subject_subcategories',
#      'project_title',
#      'project_essay_1',
#      'project_essay_2',
#      'project_essay_3',
#      'project_essay_4',
#      'project_resource_summary',
#      'teacher_number_of_previously_posted_projects'
#     ]

train = full_df[:n_train]
dev = full_df[n_train:n_train+n_dev]
test = full_df[n_train+n_dev:]

X_train = get_rnn_data(train)
Y_train = train.project_is_approved.values

X_dev = get_rnn_data(dev)
Y_dev = dev.project_is_approved.values

X_test = get_rnn_data(test)

CPU times: user 12.8 s, sys: 534 ms, total: 13.4 s
Wall time: 13.2 s


In [0]:

MAX_TEXT = Max_words
MAX_TEACHER_ID = np.max(full_df.teacher_id.max()) + 1
MAX_TEACHER_PREFIX = np.max(full_df.teacher_prefix.max()) + 1
MAX_SCHOOL_STATE = np.max(full_df.school_state.max()) + 1
MAX_GRADE = np.max(full_df.project_grade_category.max()) + 1
MAX_CATEGORIES = np.max(full_df.project_subject_categories.max()) + 1
MAX_SUBCATEGORIES = np.max(full_df.project_subject_subcategories.max()) + 1

In [0]:
full_df[:5]

,id,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_grade_category,project_is_approved,project_resource_summary,project_subject_categories,project_subject_subcategories,...,teacher_id,teacher_number_of_previously_posted_projects,teacher_prefix,sub_year,sub_month,sub_day,sub_hour,sub_minute,sub_dayofweek,sub_dayofyear
99111,p195591,"[39, 1628, 1866, 1, 79, 15, 356, 3152, 156, 66...","[15, 854, 184, 13, 23, 54, 1, 233, 5, 8638, 23...","[27, 26]","[27, 26]",1,1,"[7, 4, 28, 5, 52, 222, 6, 11429, 500, 3, 2070,...",24,142,...,116399,0,2,0,10,4,11,32,3,131
41784,p082205,"[73, 90, 578, 5, 22, 90, 1798, 595, 66, 3, 193...","[7, 22, 105, 677, 6, 2298, 53, 17, 257, 35, 48...","[27, 26]","[27, 26]",0,1,"[7, 4, 28, 2298, 53, 49, 11, 35, 145, 256, 8, ...",24,329,...,118031,10,4,0,10,9,21,15,1,137
141032,p183543,"[7, 4, 10, 40, 1750, 5, 183, 75, 22, 8, 5, 141...","[38, 20, 47, 296, 375, 1, 146, 3, 522, 732, 14...","[27, 26]","[27, 26]",3,1,"[7, 4, 28, 45, 255, 1, 23, 182, 230, 18, 610, ...",30,337,...,93862,7,3,0,1,1,9,50,5,195
157388,p098121,"[7, 4, 10, 1029, 2, 523, 37, 842, 253, 422, 10...","[404, 1, 2906, 3036, 45, 4, 1258, 52, 2, 47, 1...","[27, 26]","[27, 26]",3,1,"[7, 4, 28, 30, 2040, 175, 32, 481, 1956, 1, 64...",24,341,...,38997,2,3,0,10,12,19,32,4,140
159970,p131924,"[3, 4, 39, 7, 22, 10, 88, 6, 3, 108, 211, 444,...","[3, 449, 1581, 212, 10, 410, 120, 2445, 2, 227...","[27, 26]","[27, 26]",2,1,"[7, 4, 28, 6861, 2, 3339, 6650, 1, 68, 3550, 8...",32,353,...,73560,1,4,0,11,6,13,30,1,168


# Put a GLoVe on it!
This puts the glove word vectors into the dataset. Download the glove vectors here and path the EMBEDDING_FILE to them: https://www.kaggle.com/rtatman/glove-global-vectors-for-word-representation 

In [0]:
EMBEDDING_FILE = 'drive/Donor/glove.6B.100d.txt'

In [0]:
%%time
Embed_size = 100
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

word_index = tok_raw.word_index
nb_words = min(Max_words, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, Embed_size))
for word, i in word_index.items():
    if i >= Max_words: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

del EMBEDDING_FILE
del tok_raw

CPU times: user 16.5 s, sys: 800 ms, total: 17.3 s
Wall time: 21.8 s


# Make the model


In [0]:
# set seed again in case testing models adjustments by looping next 2 blocks
np.random.seed(123)
#  ['sub_year', 'sub_month', 'sub_day', 'sub_hour', 'sub_minute', 'sub_dayofweek', 'sub_dayofyear']
def new_rnn_model(lr=0.001, decay=0.0):
    # Inputs
    teacher_id = Input(shape=[1], name='teacher_id')
    teacher_prefix = Input(shape=[1], name='teacher_prefix')
    school_state = Input(shape=[1], name='school_state')
#     project_submitted_datetime = Input(shape=[1], name='project_submitted_datetime')
    project_grade_category = Input(shape=[1], name='project_grade_category')
    project_subject_categories = Input(shape=[1], name='project_subject_categories')
    project_subject_subcategories = Input(shape=[1], name='project_subject_subcategories')	
    project_title = Input(shape=[X_train['project_title'].shape[1]], name='project_title')	
    project_essay_1 = Input(shape=[X_train['project_essay_1'].shape[1]], name='project_essay_1')	
    project_essay_2 = Input(shape=[X_train['project_essay_2'].shape[1]], name='project_essay_2')	
    project_essay_3 = Input(shape=[X_train['project_essay_3'].shape[1]], name='project_essay_3')	
    project_essay_4 = Input(shape=[X_train['project_essay_4'].shape[1]], name='project_essay_4')	
    project_resource_summary = Input(shape=[X_train['project_resource_summary'].shape[1]], name='project_resource_summary')	
    teacher_number_of_previously_posted_projects = Input(shape=[1], name='teacher_number_of_previously_posted_projects')
    sub_year = Input(shape=[1], name='sub_year')
    sub_month = Input(shape=[1], name='sub_month')
#     sub_day = Input(shape=[1], name='sub_day')
#     sub_hour = Input(shape=[1], name='sub_hour')
#     sub_minute = Input(shape=[1], name='sub_minute')
    sub_dayofweek = Input(shape=[1], name='sub_dayofweek')
    sub_dayofyear = Input(shape=[1], name='sub_dayofyear')

    # Embeddings layers (adjust outputs to help model)
    emb_teacher_id = Embedding(MAX_TEACHER_ID, 20)(teacher_id)
    emb_teacher_prefix = Embedding(MAX_TEACHER_PREFIX, 20)(teacher_prefix)
    emb_school_state = Embedding(MAX_SCHOOL_STATE, 20)(school_state)
#     emb_project_submitted_datetime = Embedding(50, 10)(project_submitted_datetime)
    emb_project_grade_category = Embedding(MAX_GRADE, 10)(project_grade_category)
    emb_project_subject_categories = Embedding(MAX_CATEGORIES, 20)(project_subject_categories)
    emb_project_subject_subcategories = Embedding(MAX_SUBCATEGORIES, 20)(project_subject_subcategories)
    emb_project_title = Embedding(MAX_TEXT, 100, weights=[embedding_matrix], trainable=True)(project_title)	
    emb_project_essay_1 = Embedding(MAX_TEXT, 100, weights=[embedding_matrix], trainable=True)(project_essay_1)	
    emb_project_essay_2 = Embedding(MAX_TEXT, 100, weights=[embedding_matrix], trainable=True)(project_essay_2)
    emb_project_essay_3 = Embedding(MAX_TEXT, 100, weights=[embedding_matrix], trainable=True)(project_essay_3)		
    emb_project_essay_4 = Embedding(MAX_TEXT, 100, weights=[embedding_matrix], trainable=True)(project_essay_4)	
    emb_project_resource_summary = Embedding(MAX_TEXT, 100, weights=[embedding_matrix], trainable=True)(project_resource_summary)	
    emb_teacher_number_of_previously_posted_projects = Embedding(50, 10)(teacher_number_of_previously_posted_projects)
    emb_sub_year = Embedding(50, 5)(sub_year)
    emb_sub_month = Embedding(50, 12)(sub_month)
#     emb_sub_day = Embedding(50, 10)(sub_day)
#     emb_sub_hour = Embedding(50, 10)(sub_hour)
#     emb_sub_minute = Embedding(50, 10)(sub_minute)
    emb_sub_dayofweek = Embedding(50, 7)(sub_dayofweek)
    emb_sub_dayofyear = Embedding(50, 365)(sub_dayofyear)
    

    # rnn layers (GRUs are faster than LSTMs and speed is important here)
    rnn_layer1 = Bidirectional(CuDNNGRU(50)) (emb_project_title)
    rnn_layer2 = Bidirectional(CuDNNGRU(100)) (emb_project_essay_1)
    rnn_layer3 = Bidirectional(CuDNNGRU(100)) (emb_project_essay_2)
    rnn_layer4 = Bidirectional(CuDNNGRU(100)) (emb_project_essay_3)
    rnn_layer5 = Bidirectional(CuDNNGRU(100)) (emb_project_essay_4)
    rnn_layer6 = Bidirectional(CuDNNGRU(100)) (emb_project_resource_summary)

    # main layers
    main_l = concatenate([
        Flatten() (emb_teacher_id),
         Flatten() (emb_teacher_prefix)
        , Flatten() (emb_school_state)
#         , Flatten() (emb_project_submitted_datetime)
        , Flatten() (emb_project_grade_category)
        , Flatten() (emb_project_subject_categories)
        , Flatten() (emb_project_subject_subcategories)
        , Flatten() (emb_teacher_number_of_previously_posted_projects)
        , Flatten() (emb_sub_year)
        , Flatten() (emb_sub_month)
#         , Flatten() (emb_sub_day)
#         , Flatten() (emb_sub_hour)
#         , Flatten() (emb_sub_minute)
        , Flatten() (emb_sub_dayofweek)
        , Flatten() (emb_sub_dayofyear)
        , rnn_layer1
        , rnn_layer2
        , rnn_layer3
        , rnn_layer4
        , rnn_layer5
        , rnn_layer6
    ])
    # (incressing the nodes or adding layers does not effect the time quite as much as the rnn layers)
    main_l = Dropout(0.1)(Dense(512,kernel_initializer='normal',activation='elu') (main_l))
    main_l = Dropout(0.1)(Dense(256,kernel_initializer='normal',activation='elu') (main_l))
    main_l = Dropout(0.1)(Dense(128,kernel_initializer='normal',activation='elu') (main_l))
    main_l = Dropout(0.1)(Dense(64,kernel_initializer='normal',activation='elu') (main_l))

    # the output layer.
    output = Dense(1, activation="sigmoid") (main_l)
    
    model = Model([teacher_id,
     teacher_prefix,
     school_state,
#      project_submitted_datetime,
     project_grade_category,
     project_subject_categories,
     project_subject_subcategories,
     project_title,
     project_essay_1,
     project_essay_2,
     project_essay_3,
     project_essay_4,
     project_resource_summary,
     teacher_number_of_previously_posted_projects,
     sub_year,
     sub_month,
#      sub_day,
#      sub_hour,
#      sub_minute,
     sub_dayofweek,
     sub_dayofyear
    ], output)

    optimizer = Adam(lr=lr, decay=decay)
    # (mean squared error loss function works as well as custom functions)  
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics=["accuracy"])

    return model

model = new_rnn_model()
model.summary()
del model

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
teacher_id (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
teacher_prefix (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
school_state (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
project_grade_category (InputLa (None, 1)            0                                            
__________________________________________________________________________________________________
project_su

In [0]:
# Get a checkpoint going
file_path="drive/Donor/weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
model_callbacks = [checkpoint]

# Fit the Model

In [0]:
%%time

# Set hyper parameters for the model.
BATCH_SIZE = 32
epochs = 2

# Calculate learning rate decay.
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(X_train['teacher_id']) / BATCH_SIZE) * epochs
lr_init, lr_fin = 0.005, 0.001
lr_decay = exp_decay(lr_init, lr_fin, steps)

# Create model and fit it with training dataset.
rnn_model = new_rnn_model(lr=lr_init, decay=lr_decay)
rnn_model.fit(
        X_train, Y_train,
        epochs=epochs,
        batch_size=BATCH_SIZE,
        validation_data=(X_dev, Y_dev),
        verbose=1,
        callbacks = model_callbacks,
)

Train on 163872 samples, validate on 18208 samples
Epoch 1/2
 21440/163872 [==>...........................] - ETA: 19:49 - loss: 0.4490 - acc: 0.8469

163840/163872 [============================>.] - ETA: 0s - loss: 0.4347 - acc: 0.8470

163872/163872 [==============================] - 1379s 8ms/step - loss: 0.4346 - acc: 0.8470 - val_loss: 0.4236 - val_acc: 0.8503

Epoch 00001: val_loss improved from inf to 0.42356, saving model to drive/Donor/weights_base.best.hdf5
Epoch 2/2
  7552/163872 [>.............................] - ETA: 21:33 - loss: 0.4297 - acc: 0.8481

163840/163872 [============================>.] - ETA: 0s - loss: 0.4301 - acc: 0.8474

163872/163872 [==============================] - 1372s 8ms/step - loss: 0.4301 - acc: 0.8474 - val_loss: 0.4246 - val_acc: 0.8503

Epoch 00002: val_loss did not improve
CPU times: user 33min 37s, sys: 16min 42s, total: 50min 19s
Wall time: 47min 58s


# Prediction

In [0]:
# import keras as K
# fmodel = K.models.load_model(file_path)

In [0]:
%%time

Y_test = rnn_model.predict(X_test)


CPU times: user 2min 37s, sys: 1min 19s, total: 3min 56s
Wall time: 3min 32s


In [0]:
Y_test

array([[0.83546454],
       [0.83546454],
       [0.83546454],
       ...,
       [0.83546454],
       [0.83546454],
       [0.83546454]], dtype=float32)

# Make the submission

In [0]:
submission = pd.read_csv("drive/Donor/sample_submission.csv")
submission['project_is_approved'] = Y_test
submission.to_csv("drive/Donor/CuDNN_Donor_Submission2.csv", index=False)


In [0]:
# rnn_model.load_weights(file_path)
rnn_model.save('drive/Donor/my_model.h5')